In [ ]:
# Visualizing policy using method from https://arxiv.org/abs/1711.00138
# GitHub repo: https://github.com/greydanus/visualize_atari
# Imports
import os
import sys
import pdb
import json
import torch
import random
import argparse
import torchvision
import tensorboardX
import torch.optim as optim
import torchvision.utils as vutils
import matplotlib.pyplot as plt
import skimage.transform as skitran

sys.path.append(os.path.join(os.path.abspath(''), 'misc/'))

from utils import *
from models import *
from torch.optim import lr_scheduler
from tensorboardX import SummaryWriter
from scipy.ndimage.filters import gaussian_filter

In [ ]:
# Refer https://github.com/greydanus/visualize_atari/blob/master/saliency.py

def occlude(I, mask):
    num_channels = I.shape[2]
    I_out = np.zeros(I.shape)
    for chn in range(num_channels):
        I_out[:, :, chn] = I[:, :, chn]*(1-mask) + gaussian_filter(I[:, :, chn], sigma=5)*mask 
    return I_out
    
def get_mask(center, size, r):
    y,x = np.ogrid[-center[0]:size[0]-center[0], -center[1]:size[1]-center[1]]
    keep = x*x + y*y <= 1
    mask = np.zeros(size) ; mask[keep] = 1 # select a circle of pixels
    mask = gaussian_filter(mask, sigma=r) # blur the circle of pixels. this is a 2D Gaussian for r=r^2=1
    return mask/mask.max()

def saliency_on_image(saliency, image, fudge_factor, channels=[2], sigma=0):
    # Refer https://github.com/greydanus/visualize_atari/blob/master/saliency.py
    pmax = saliency.max()
    S = saliency
    S = S if sigma == 0 else gaussian_filter(S, sigma=sigma)
    S -= S.min()
    S = 255.0*fudge_factor*pmax*S/S.max()
    I = np.copy(image.astype('uint16'))
    if image.shape[2] == 1:
        I[:, :, 0] += S.astype('uint16')
    else:
        for channel in channels:
            I[:, :, channel] += S.astype('uint16')
    I = I.clip(1, 255).astype('uint8')
    return I

def get_belief_saliency(belief, others, agent, iscuda, lr=1e-4, weight_decay=1e-1, iters=200):
    """
    belief: current hidden state of the aggregator
    others: dictionary of elevation('elev'), azimuth('azim'), time('time'), proprioception change('pro')
    NOTE: Assuming batch size = 1
          Ensure that the agent's parameters have requires_grad=False
    """
    M = agent.M
    N = agent.N
    C = agent.C
    d_belief_orig = Variable(torch.randn(1, 256)*1e-3, requires_grad=True)
    optimizer = optim.SGD([d_belief_orig], lr=lr, weight_decay=weight_decay, momentum=0.9)
    
    act_input_1 = torch.cat([belief.view(1, -1), others['pro'][:, :2]], dim=1) 
    
    if 'elev' in others:
        xe = others['elev']
        act_input_1 = torch.cat([act_input_1, xe], dim=1)
    if 'azim' in others:
        xa = others['azim']
        act_input_1 = torch.cat([act_input_1, xa], dim=1)
    if 'time' in others:
        xt = others['time']
        act_input_1 = torch.cat([act_input_1, xt], dim=1)
    
    activations_1 = agent.policy.act(act_input_1)
    # Solve for d_belief that maximises change in policy activations
    for i in range(iters):
        if iscuda:
            d_belief = d_belief_orig.cuda()
        else:
            d_belief = d_belief_orig
            
        act_input_2 = torch.cat([d_belief + belief.view(1, -1), others['pro'][:, :2]], dim=1)
        
        if 'elev' in others:
            xe = others['elev']
            act_input_2 = torch.cat([act_input_2, xe], dim=1)
        if 'azim' in others:
            xa = others['azim']
            act_input_2 = torch.cat([act_input_2, xa], dim=1)
        if 'time' in others:
            xt = others['time']
            act_input_2 = torch.cat([act_input_2, xt], dim=1)

        activations_2 = agent.policy.act(act_input_2)
        loss = -((activations_1 - activations_2)**2).sum() + weight_decay*(d_belief**2).sum()
        loss.backward()
        #pdb.set_trace()
        optimizer.step()
        if torch.norm(d_belief_orig.data) >= 0.75 * torch.norm(belief.data):
            break
    diff = -((activations_1 - activations_2)**2).sum()
    diff_probs = -((F.softmax(activations_1, dim=1) - F.softmax(activations_2, dim=1))**2).sum()
    #pdb.set_trace()
    #print('Final loss: %.3f, Act prob diff: %f, Belief norm: %.4f, Delta norm: %.4f, Terminated iter: %d'%\
    #                 (diff.data[0], diff_probs.data[0], torch.norm(belief)[0], torch.norm(d_belief_orig)[0], i)) 
    
    # Get the saliency
    decoded_2 = agent.policy.decode(F.normalize(d_belief + belief.view(1, -1), p=1, dim=1))
    decoded_2 = decoded_2.view(1, N, M, C, 32, 32)
    decoded_1 = agent.policy.decode(F.normalize(belief.view(1, -1), p=1, dim=1))
    decoded_1 = decoded_1.view(1, N, M, C, 32, 32)
    saliency = np.sum(0.5*((decoded_1 - decoded_2).data.cpu().numpy())**2, axis=3)
    #saliency /= (saliency.max()+1e-9)
    
    return saliency, diff.data[0]

In [ ]:
model_path = '/vision/vision_users/srama/Research/LearningToLookAround/models/SUN360_models_very_old/multi_no_expert_reinforce/run1/model_best.net'
loaded_state = torch.load(model_path)
opts = loaded_state['opts']
opts.batch_size = 50
opts.critic_full_obs = False
opts.act_full_obs = False

if opts.dataset == 0:
    opts.h5_path = '/vision/vision_users/srama/Research/LearningToLookAround/data/SUN360/data_4_plus_highres.h5'
    
if not hasattr(opts, 'h5_path_unseen'):
    if opts.dataset == 0:
        opts.h5_path_unseen = ''
    else:
        opts.h5_path_unseen = ' ../data/ModelNet/data_10.h5'
        
if opts.expert_trajectories:
    opts.T_sup = 3

if opts.expert_trajectories:
    agent = AgentSupervised(opts)
else:
    agent = Agent(opts)
    
agent.policy.load_state_dict(loaded_state['state_dict'])
agent.policy.eval()

opts.expert_rewards = False
opts.expert_trajectories = False

from DataLoader import DataLoaderSimple as DataLoader

loader = DataLoader(opts)

set_random_seeds(opts.seed)

if opts.dataset == 0:
    opts.num_channels = 3
    if opts.mean_subtract:
        # R, G, B means and stds
        opts.mean = [119.16, 107.68, 95.12]
        opts.std = [61.88, 61.72, 67.24]
    else:
        opts.mean = [0, 0, 0]
        opts.std = [1, 1, 1]
elif opts.dataset == 1:
    opts.num_channels = 1
    if opts.mean_subtract:
        # R, G, B means and stds
        opts.mean = [193.0162338615919]
        opts.std = [37.716024486312811]
    else:
        opts.mean = [0]
        opts.std = [1]
else:
    raise ValueError('Dataset %d does not exist!'%(opts.dataset))
    
# Avoid computing gradients for the agent in backward
for parameter in agent.policy.parameters():
    parameter.requires_grad = False

In [ ]:
# Disable this if no need for highres
opts.highres = False

saliency_scores_all = []
saliency_belief_all = []
images_all = []
pano_all = []
if opts.highres:
    images_all_highres = []
    
decoded_all = []
decoded_saliency_all = []
decoded_all_raw = []

num_iters = 4

for iters in range(num_iters):
    if opts.highres:
        pano, pano_highres, _, depleted = loader.next_batch_test(highres=True)
    else:
        pano, _, depleted = loader.next_batch_test(highres=False)
    
    pano_rewards = None
    pano_maps = None
    
    batch_size = opts.batch_size
    start_idx = get_starts(opts.N, opts.M, batch_size, opts.start_view)
    state_object = State(pano, pano_rewards, start_idx, opts)
    hidden = None
    # Collect data to display
    images_all.append([])
    saliency_scores_all.append([])
    decoded_all.append([])
    saliency_belief_all.append([])
    decoded_all_raw.append([])
    visited_idxes = []
    decoded_saliency_all.append([])    
    if opts.highres:
        images_all_highres.append([])
        
    # Get the panoramas
    pano_full_view = np.copy(state_object.views)
    if opts.highres:
        pano_full_view_highres = np.copy(pano_highres)
    for i in range(batch_size):
        if opts.num_channels == 3:
            pano_full_view[i, state_object.start_idx[i][0], state_object.start_idx[i][1], 0, :3, :] = 255
            pano_full_view[i, state_object.start_idx[i][0], state_object.start_idx[i][1], 0, :, :3] = 255
            pano_full_view[i, state_object.start_idx[i][0], state_object.start_idx[i][1], 0, -3:, :] = 255
            pano_full_view[i, state_object.start_idx[i][0], state_object.start_idx[i][1], 0, :, -3:] = 255

            pano_full_view[i, state_object.start_idx[i][0], state_object.start_idx[i][1], 1:, :3, :] = 0
            pano_full_view[i, state_object.start_idx[i][0], state_object.start_idx[i][1], 1:, :, :3] = 0
            pano_full_view[i, state_object.start_idx[i][0], state_object.start_idx[i][1], 1:, -3:, :] = 0
            pano_full_view[i, state_object.start_idx[i][0], state_object.start_idx[i][1], 1:, :, -3:] = 0
           
        else:
            pano_full_view[i, state_object.start_idx[i][0], state_object.start_idx[i][1], 0, :3, :] = 0
            pano_full_view[i, state_object.start_idx[i][0], state_object.start_idx[i][1], 0, :, :3] = 0
            pano_full_view[i, state_object.start_idx[i][0], state_object.start_idx[i][1], 0, -3:, :] = 0
            pano_full_view[i, state_object.start_idx[i][0], state_object.start_idx[i][1], 0, :, -3:] = 0
            
    pano_full_view = pano_full_view.transpose((0, 3, 1, 4, 2, 5)).reshape(\
                                batch_size, 1, opts.num_channels, opts.N*32, opts.M*32)
    pano_all.append(pano_full_view)
    
    for t in range(opts.T):
        # Get the original image's action distribution
        im_np, pro = state_object.get_view() # im_np - BxCx32x32
        if opts.highres:
            state_idx = state_object.idx
            # use python advanced indexing to get high resolution images of current views
            im_np_highres = pano_highres[range(len(state_idx)), [idx_[0] for idx_ in state_idx], \
                                                                [idx_[1] for idx_ in state_idx]]
            
        im, pro = preprocess(im_np, pro)
        
        # Store the visited locations
        visited_idxes.append(state_object.idx)
        
        C = im.shape[1]
        W = im.shape[2]
        H = im.shape[3]

        policy_input = {'im': im, 'pro': pro}
        # Assuming only actOnElev
        policy_input['elev'] = torch.Tensor([[state_object.idx[i][0]] for i in range(batch_size)])
        policy_input['time'] = torch.Tensor([[t] for i in range(batch_size)])
        if opts.iscuda:
            for var in policy_input:
                policy_input[var] = policy_input[var].cuda()
        
        for var in policy_input:
            policy_input[var] = Variable(policy_input[var])
        
        probs, decoded, hidden_new, value = agent.policy.forward(policy_input, hidden)
        saliency_scores = np.zeros((batch_size, H, W))
        # ============ Perturb the image at different pixel locations ============ 
        for loc_x in range(0, W):
            for loc_y in range(0, H):
                
                mask = get_mask(center=[loc_y,loc_x], size=[32,32], r=3)
                
                # To check if mask is working
                #if loc_y == H // 2:
                #if loc_x % 4 == 0 and loc_y % 4 == 0:
                #    plt.imshow(mask)
                #    plt.show()
                #pdb.set_trace()
                
                im_ptbd_all = np.zeros(im_np.shape)
                for i in range(batch_size):
                    im_ptbd = occlude(np.transpose(im_np[i], (1, 2, 0)),mask)
                    im_ptbd_all[i] = np.transpose(im_ptbd, (2, 0, 1))
                policy_input['im'] = Variable(torch.Tensor(im_ptbd_all))
                if opts.iscuda:
                    policy_input['im'] = policy_input['im'].cuda()
                
                probs_x_y, _, _, value_x_y = agent.policy.forward(policy_input, hidden)
                saliency_scores[:, loc_y, loc_x] = (probs-probs_x_y).pow(2).sum(dim=1).cpu().data.numpy()
        #pdb.set_trace()

        for i in range(batch_size):
            saliency_scores[i] /= (saliency_scores[i].max() + 1e-10)
        im_np_orig = np.array(im_np)*255.0
        for chn in range(len(opts.mean)):
            im_np_orig[:, chn] += opts.mean[chn]
            
        images_all[iters].append(im_np_orig)
        saliency_scores_all[iters].append(saliency_scores)
        if opts.highres:
        # Apply scaled up saliency scores on the high resolution images
            for i in range(batch_size):
                saliency_score_highres = skitran.resize(saliency_scores[i], (448, 448))
                im_np_curr = np.transpose(im_np_highres[i], (1, 2, 0))
                im_np_curr = saliency_on_image(saliency_score_highres, im_np_curr, 1, \
                                                                             channels=[0, 2], sigma=2)
                im_np_highres[i] = np.transpose(im_np_curr, (2, 0, 1))

            images_all_highres[iters].append(im_np_highres)    
            
        # Create the decoded image with saliency modified view filled in
        decoded_images = decoded.data.cpu().numpy()*255.0
        # Add the means
        for chn in range(len(opts.mean)):
            decoded_images[:, :, :, chn] += opts.mean[chn]
            
        decoded_images_shifted = np.zeros(decoded_images.shape)
        decoded_images_shifted_wo_sal = np.zeros(decoded_images.shape)
        
        for i in range(batch_size):
            decoded_images_shifted[i] = np.copy(np.roll(decoded_images[i], state_object.start_idx[i][1], axis=1))
            decoded_images_shifted_wo_sal[i] = np.copy(np.roll(decoded_images[i], state_object.start_idx[i][1], axis=1))
            
        for t_view, visited_view in enumerate(visited_idxes):
            for i in range(batch_size):                
                curr_view = np.copy(state_object.views[i, visited_view[i][0], visited_view[i][1]])
                curr_view = np.transpose(curr_view, (1, 2, 0))
                # Modify the current view to have some borders (for easy visualization)
                if t_view == t:
                    if curr_view.shape[2] == 1:
                        # If single channeled
                        curr_view[:3, :, :] = 255 
                        curr_view[-3:, :, :] = 255
                        curr_view[:, :3, :] = 255
                        curr_view[:, -3:, :] = 255
                    else:
                        # If 3 channeled
                        curr_view[:3, :, 0] = 255
                        curr_view[-3:, :, 0] = 255
                        curr_view[:, :3, 0] = 255
                        curr_view[:, -3:, 0] = 255
                        curr_view[:3, :, 1:] = 0 
                        curr_view[-3:, :, 1:] = 0
                        curr_view[:, :3, 1:] = 0
                        curr_view[:, -3:, 1:] = 0
                    
                curr_saliency = saliency_scores_all[iters][t_view][i]
                curr_view_saliency = saliency_on_image(curr_saliency, curr_view, 1, channels=[0, 2], sigma=2)
                decoded_images_shifted[i, visited_view[i][0], visited_view[i][1]] = \
                    np.transpose(curr_view_saliency, (2, 0, 1))
                    
                decoded_images_shifted_wo_sal[i, visited_view[i][0], visited_view[i][1]] = \
                    np.transpose(curr_view, (2, 0, 1))
        
        # Convert BxNxMxCx32x32 -> BxCx(32*N)x(32*M)
        decoded_image_full_view = decoded_images_shifted.transpose((0, 3, 1, 4, 2, 5)).reshape(\
                                            batch_size, 1, opts.num_channels, opts.N*32, opts.M*32)
        decoded_image_full_view_wo_sal = decoded_images_shifted_wo_sal.transpose((0, 3, 1, 4, 2, 5)).reshape(\
                                            batch_size, 1, opts.num_channels, opts.N*32, opts.M*32)
        
        decoded_all[iters].append(decoded_image_full_view)
        decoded_all_raw[iters].append(decoded_image_full_view_wo_sal)
        
        # ============ Get the belief state saliency ============ 
        saliency_belief = []
        for i in range(batch_size):
            policy_input_curr = {}
            # Get only current batch element
            for k, v in policy_input.iteritems():
                policy_input_curr[k] = v[i:i+1]
            saliency_belief_curr, _ = get_belief_saliency(hidden_new[0][:, i:i+1], policy_input_curr, agent, opts.iscuda)
            # roll saliency_belief_curr to reflect the unknown azimuth
            saliency_belief_curr = np.roll(saliency_belief_curr, state_object.start_idx[i][1], axis=2)
            saliency_belief_curr = saliency_belief_curr.transpose((0, 1, 3, 2, 4)).reshape(1, opts.N*32, opts.M*32)
            saliency_belief.append(saliency_belief_curr)
        
        saliency_belief = np.concatenate(saliency_belief, axis=0)
        saliency_belief_all[iters].append(saliency_belief)
                        
        # Act greedily
        _, act = probs.max(dim=1)
        act = act.data.view(-1, 1)
        # Rotate the view
        _ = state_object.rotate(act[:, 0])
        
        # Set hidden
        hidden = hidden_new
    
    # Normalize the saliency values
    for i in range(batch_size):        
        saliency_max = 0
        # Across time
        #for t in range(opts.T):
        #    saliency_curr = saliency_belief_all[iters][t][i]
        #    curr_max = saliency_curr.max()
        #    if curr_max > saliency_max:
        #        saliency_max = curr_max
        for t in range(opts.T):
            # If normalizing only wrt time t
            saliency_max = saliency_belief_all[iters][t][i].max()
            # Normalize
            saliency_belief_all[iters][t][i] /= (saliency_max + 1e-9)
    
    # Impose saliency on decoded views
    for t in range(opts.T):

        decoded_saliency_full_view = np.zeros((batch_size, 1, opts.num_channels, opts.N*32, opts.M*32))
        for i in range(batch_size):
            curr_saliency = saliency_belief_all[iters][t][i]
            curr_view = decoded_all_raw[iters][t][i, 0].transpose(1, 2, 0)
            decoded_saliency_full_view[i, 0] = saliency_on_image(curr_saliency, curr_view, 1, channels=[0, 2], sigma=2).transpose(2, 0, 1)
        
        decoded_saliency_all[iters].append(decoded_saliency_full_view)    

In [ ]:
# Visualize image saliency in numpy 
for iters in range(1):
    batch_size = images_all[iters][0].shape[0]
    images_count = 1
    
    for i in range(batch_size):
        display_image = np.transpose(pano_all[iters][i][0], (1, 2, 0)).astype('float32')
        display_image = (display_image - display_image.min())/(display_image.max()-display_image.min())
        fig = plt.figure(figsize=(12, 16))
        if display_image.shape[2] == 1:
            plt.imshow(display_image[:, :, 0])
        else:
            plt.imshow(display_image)
        plt.show()
        for t in range(opts.T):
            #fig.add_subplot(batch_size*opts.T, 1, images_count)
            fig = plt.figure(figsize=(12, 16))
            display_image = np.transpose(decoded_all[iters][t][i][0], (1, 2, 0))
            display_image = (display_image - display_image.min())/(display_image.max()-display_image.min())
            if display_image.shape[2] == 1:
                plt.imshow(display_image[:, :, 0])
            else:
                plt.imshow(display_image)
            images_count += 1
            plt.show()
            if opts.highres:
                fig = plt.figure(figsize=(12, 16))
                display_image = np.transpose(images_all_highres[iters][t][i], (1, 2, 0)).astype(np.float32)
                display_image = (display_image - display_image.min())/(display_image.max()-display_image.min())
                if display_image.shape[2] == 1:
                    plt.imshow(display_image[:, :, 0])
                else:
                    plt.imshow(display_image)
                plt.show()
            
            

In [ ]:
# Write outputs to tensorboard
save_path = 'visualizations/vis_test/SUN360/multi_no_expert_reinforce/'
writer = SummaryWriter(log_dir=save_path)
images_count = 0
for iters in range(num_iters):
    batch_size = images_all[iters][0].shape[0]
    for i in range(batch_size):
        images_count += 1
        outputs_all = []
        display_image = np.transpose(pano_all[iters][i], (0, 2, 3, 1)).astype('float32')
        display_image = (display_image - display_image.min())/(display_image.max()-display_image.min())
        outputs_all.append(display_image)
        for t in range(opts.T):
            display_image = np.transpose(decoded_all[iters][t][i], (0, 2, 3, 1))
            display_image = (display_image - display_image.min())/(display_image.max()-display_image.min())
            outputs_all.append(display_image)
            
        outputs_all = np.concatenate(outputs_all, axis=0).transpose((0, 3, 1, 2))
        x = vutils.make_grid(torch.Tensor(outputs_all), padding=5, normalize=True, scale_each=True, nrow=opts.T+1, pad_value=1.0)
        writer.add_image('#%d Image Saliency'%(images_count), x, 0)


In [ ]:
# Visualize belief saliency in numpy
for iters in range(1):
    batch_size = decoded_saliency_all[iters][0].shape[0]
    for i in range(batch_size):
        for t in range(opts.T):
            #pdb.set_trace()
            display_image = np.transpose(decoded_saliency_all[iters][t][i][0], (1, 2, 0)).astype('float32')
            display_image = (display_image - display_image.min())/(display_image.max()-display_image.min())
            fig = plt.figure(figsize=(12, 16))
            if display_image.shape[2] == 1:
                plt.imshow(display_image[:, :, 0])
            else:
                plt.imshow(display_image)
            plt.show()

In [ ]:
# Write outputs to tensorboard
images_count = 0
for iters in range(num_iters):
    batch_size = decoded_saliency_all[iters][0].shape[0]
    for i in range(batch_size):
        images_count += 1
        outputs_all = []
        display_image = pano_all[iters][i].astype('float32')
        display_image = (display_image - display_image.min())/(display_image.max()-display_image.min())
        outputs_all.append(display_image)
        for t in range(opts.T):
            display_image = decoded_saliency_all[iters][t][i].astype('float32')
            display_image = (display_image - display_image.min())/(display_image.max()-display_image.min())
            outputs_all.append(display_image)
        
        outputs_all = np.concatenate(outputs_all, axis=0)
        x = vutils.make_grid(torch.Tensor(outputs_all), padding=5, normalize=True, scale_each=True, nrow=opts.T+1, pad_value=1.0)
        writer.add_image('#%d Belief Saliency'%(images_count), x, 0)


writer.close()

In [ ]:
num_iters = 1
loader.val_idx = 0
for iters in range(num_iters):
    if opts.expert_rewards:
        pano, pano_rewards, depleted = loader.next_batch_val()
        pano_maps = None
    elif opts.expert_trajectories:
        pano, pano_maps, depleted = loader.next_batch_val()
        pano_rewards = None #TODO: Add expert rewards
    else:
        pano, depleted = loader.next_batch_val()
        pano_rewards = None
        pano_maps = None
    
    batch_size = opts.batch_size
    start_idx = get_starts(opts.N, opts.M, batch_size, opts.start_view)
    state_object = State(pano, pano_rewards, start_idx, opts)
    hidden = None
    
    for b in range(batch_size):
        for t in range(opts.T):
            print('============ t = %d ================='%(t))
            # Get the original image's action distribution
            
            im_np, pro = state_object.get_view() # im_np - BxCx32x32
            im, pro = preprocess(im_np, pro)

            C = im.shape[1]
            W = im.shape[2]
            H = im.shape[3]

            policy_input = {'im': im, 'pro': pro}
            # Assuming only actOnElev
            policy_input['elev'] = torch.Tensor([[state_object.idx[i][0]] for i in range(batch_size)])
            policy_input['time'] = torch.Tensor([[t] for i in range(batch_size)])
            if opts.iscuda:
                for var in policy_input:
                    policy_input[var] = policy_input[var].cuda()

            for var in policy_input:
                policy_input[var] = Variable(policy_input[var])

            probs, decoded, hidden_new, value = agent.policy.forward(policy_input, hidden)

            # ============ Get the belief state saliency ============ 
            saliency_belief = []
            for i in [b]:
                print('Original panorama')
                pano_copy = np.array(state_object.views[b])
                pano_copy[state_object.idx[b][0], state_object.idx[b][1], 0, -3:, :] = 255
                pano_copy[state_object.idx[b][0], state_object.idx[b][1], 0, :, -3:] = 255
                pano_copy[state_object.idx[b][0], state_object.idx[b][1], 0, :3, :] = 255
                pano_copy[state_object.idx[b][0], state_object.idx[b][1], 0, :, :3] = 255
                
                pano_copy[state_object.idx[b][0], state_object.idx[b][1], 1, -3:, :] = 0
                pano_copy[state_object.idx[b][0], state_object.idx[b][1], 1, :, -3:] = 0
                pano_copy[state_object.idx[b][0], state_object.idx[b][1], 1, :3, :] = 0
                pano_copy[state_object.idx[b][0], state_object.idx[b][1], 1, :, :3] = 0
                
                pano_copy[state_object.idx[b][0], state_object.idx[b][1], 2, -3:, :] = 0
                pano_copy[state_object.idx[b][0], state_object.idx[b][1], 2, :, -3:] = 0
                pano_copy[state_object.idx[b][0], state_object.idx[b][1], 2, :3, :] = 0
                pano_copy[state_object.idx[b][0], state_object.idx[b][1], 2, :, :3] = 0
                
                plt.imshow(pano_copy.transpose((2, 0, 3, 1, 4)).reshape(\
                                            opts.num_channels, opts.N*32, opts.M*32)\
                                            .transpose(1, 2, 0))
                plt.show()
                for settings in [{'lr': 1e-4, 'iters': 2000}, {'lr': 1e-3, 'iters': 2000}, \
                                 {'lr': 1e-2, 'iters': 2000}, {'lr': 1e-1, 'iters': 2000}, \
                                 {'lr': 1e-4, 'iters': 20}, {'lr': 1e-3, 'iters': 20}, \
                                 {'lr': 1e-2, 'iters': 20}, {'lr': 1e-1, 'iters': 20}]:

                    policy_input_curr = {}
                    # Get only current batch element
                    for k, v in policy_input.iteritems():
                        policy_input_curr[k] = v[i:i+1]
                    saliency_belief_curr, loss_value = get_belief_saliency(\
                                            hidden_new[0][:, i:i+1], policy_input_curr, \
                                            agent, opts.iscuda, lr=settings['lr'], \
                                            iters=settings['iters'])
                    # roll saliency_belief_curr to reflect the unknown azimuth
                    saliency_belief_curr = np.roll(saliency_belief_curr, state_object.start_idx[i][1], axis=2)
                    saliency_belief_curr = saliency_belief_curr.transpose((0, 1, 3, 2, 4)).reshape(1, opts.N*32, opts.M*32)
                    print('Settings: LR %.5f, Iters: %d, Loss: %f'%(settings['lr'], settings['iters'], loss_value))
                    fig = plt.figure(figsize=(3, 4))
                    plt.imshow(saliency_belief_curr[0])
                    plt.show()
                pdb.set_trace()

            # Act greedily
            _, act = probs.max(dim=1)
            act = act.data.view(-1, 1)
            # Rotate the view
            _ = state_object.rotate(act[:, 0])

            # Set hidden
            hidden = hidden_new
    #pdb.set_trace()